# Recurrent PPO

### Let's run Recurrent PPO on LunarLander!

#### Imports

In [1]:
from optimrl.algorithms.ppo import PPOLossFunction, PPOOptimizer

import gym
import torch
import random
import torch.nn as nn
import torch.nn.functional as F
import torch.distributions as td
import numpy as np
from tqdm import tqdm

#### Seeds

In [2]:
SEED = 1
EVAL_SEED = 2

random.seed(SEED)
np.random.seed(SEED)
_ = torch.manual_seed(SEED)

#### Grab obs + act dims

In [3]:
env = gym.make("LunarLander-v2")
obs_dims = env.observation_space.shape[-1]
act_dims = env.action_space.n
print(f"Obs dims: {obs_dims}, Action dims: {act_dims}")

Obs dims: 8, Action dims: 4


### Create policy

In [4]:
from typing import Dict, Any

def layer_init(layer, std=np.sqrt(2), bias_const=0.0):
    torch.nn.init.orthogonal_(layer.weight, std)
    torch.nn.init.constant_(layer.bias, bias_const)
    return layer

class PPORecurrentCategoricalPolicy(nn.Module):
    def __init__(
        self,
        obs_dims,
        act_dims,
        obs_hidden_dim: int = 256,
        gru_hidden_dim: int = 128
    ):
        super().__init__()
        self.obs_dims = obs_dims
        self.act_dims = act_dims
        self.obs_hidden_dim = obs_hidden_dim
        self.gru_hidden_dim = gru_hidden_dim

        self.backbone = nn.Sequential(
            layer_init(nn.Linear(self.obs_dims, obs_hidden_dim)),
            nn.ReLU(),
            layer_init(nn.Linear(obs_hidden_dim, obs_hidden_dim)),
            nn.ReLU(),
        )
        self.gru = nn.GRUCell(obs_hidden_dim, self.gru_hidden_dim)
        for name, param in self.gru.named_parameters():
            if "bias" in name:
                nn.init.constant_(param, 0)
            elif "weight" in name:
                nn.init.orthogonal_(param, 1.0)

        self.critic = nn.Sequential(
            layer_init(nn.Linear(self.gru_hidden_dim, 1)),
        )
        self.actor = nn.Sequential(
            layer_init(nn.Linear(self.gru_hidden_dim, self.act_dims)),
        )

    def forward(self, obs: torch.Tensor, policy_out = {}):
        if "hidden" not in policy_out:
            policy_out["hidden"] = torch.zeros((obs.shape[0], self.gru_hidden_dim)).to(obs.device)
        backbone = self.backbone(obs)
        hidden = self.gru(backbone, policy_out["hidden"]).view(obs.shape[0], -1)
        logits = self.actor(hidden)
        values = self.critic(hidden)
        dist = td.Categorical(logits=logits)
        actions = dist.sample()
        entropy = dist.entropy()
        log_probs = dist.log_prob(actions)
        return {
            "actions":actions,
            "values":values,
            "log_probs":log_probs,
            "dist":dist,
            "logits":logits,
            "entropy":entropy,
            "hidden":hidden
        }

    def train_forward(
        self, obs: torch.Tensor, policy_out: Dict[str, Any] = {}
    ) -> Dict[str, Any]:
        num_steps = obs.shape[0]
        num_envs = obs.shape[1]
        logits = []
        values = []
        for i in range(num_steps):
            policy_out = self.forward(obs=obs[i], policy_out=policy_out)
            logits.append(policy_out["logits"])
            values.append(policy_out["values"])
        logits = torch.stack(logits).view(num_steps*num_envs, -1)
        values = torch.stack(values).view(num_steps*num_envs, -1)
        dist = td.Categorical(logits=logits)
        return {"dist":td.Categorical(logits=logits), "entropy": dist.entropy(), "values":values}

    def act(self, obs: torch.Tensor, policy_out = {}):
        with torch.no_grad():
            out = self.forward(torch.from_numpy(obs), policy_out)
            return out["actions"].detach().cpu().numpy(), out

#### Train

##### Setup optimizer

In [5]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
policy = PPORecurrentCategoricalPolicy(8, 4)
loss_fn = PPOLossFunction(clip_ratio=0.2, ent_coef=0.01, vf_coef=1.0)
optimizer = PPOOptimizer(policy, loss_fn, pi_lr=0.0002, n_updates=5, num_minibatches=4, recurrent=True)

policy = policy.to(device)
optimizer = optimizer.to(device)

In [6]:
# obs = torch.zeros((100,4,8))
# out = policy.train_forward(obs)

##### Create vector envs

In [7]:
def make_env(env_name):
    env =  gym.make(env_name)
    env = gym.wrappers.RecordEpisodeStatistics(env)
    return env

train_envs = gym.vector.SyncVectorEnv(
    [lambda: make_env("LunarLander-v2") for i in range(4)],
)
test_envs = gym.vector.SyncVectorEnv(
    [lambda: make_env("LunarLander-v2") for i in range(1)],
)

##### Main loop

In [8]:
NUM_STEPS = 20000
MAX_ENV_STEPS = 128

In [9]:
mean_train_rewards = []
mean_test_rewards = []
bar = tqdm(np.arange(NUM_STEPS))

for i in bar:
    with torch.no_grad():
        train_rollout = optimizer.rollout(train_envs, policy, MAX_ENV_STEPS, SEED, evaluate=False)
        eval_rollout = optimizer.rollout(test_envs, policy, MAX_ENV_STEPS, EVAL_SEED, evaluate=True)
    loss, rewards, stats = optimizer.step(
        train_rollout.to_torch(
            device=optimizer.device
        )
    )
    mean_train_rewards.append(train_rollout.stats["sum_rewards"])
    mean_test_rewards.append(eval_rollout.stats["sum_rewards"])
    if i % 50 == 0:
        print(f"Train: {np.mean(mean_train_rewards[-20:])} Test: {np.mean(mean_test_rewards[-20:])}")

  0%|                  | 0/20000 [00:00<?, ?it/s]/Users/shyam/anaconda3/envs/py310/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
  0%|        | 1/20000 [00:01<5:36:35,  1.01s/it]

Train: -283.38802796338604 Test: -351.7716423374975


  0%|       | 51/20000 [00:51<5:30:49,  1.01it/s]

Train: -220.67450572555362 Test: -183.13972498388426


  1%|      | 101/20000 [01:42<5:40:51,  1.03s/it]

...

  9%|▍    | 1701/20000 [29:42<5:33:27,  1.09s/it]

Train: 88.57169737394683 Test: 97.37401649312261


  9%|▍    | 1751/20000 [30:36<5:37:31,  1.11s/it]

Train: 110.41773025592654 Test: 102.14071884415799


  9%|▍    | 1752/20000 [30:38<5:19:07,  1.05s/it]

KeyboardInterrupt

